In [75]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import json
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [76]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer


ruta = "F:/nuevo_conocimiento/CODIGO DE TESIS (MUY IMPORTANTE)/Experimentos_con_modelos_basicos/dataset_preprocesado/dataset_abstract_preprocesado.csv"

datos = pd.read_csv(ruta)
datos

,title,abstract,categorical,categoria,cat_one_hot,abstract_limpio,abstract_preprocesado
0,Electromagnetic neutrinos in laboratory experi...,An overview of neutrino electromagnetic proper...,astrophysics,humano,"[1, 0, 0, 0, 0]",An overview of neutrino electromagnetic proper...,"['overview', 'neutrino', 'electromagnetic', 'p..."
1,Infrared complex refractive index of astrophys...,In dense and cold regions of the interstellar ...,astrophysics,humano,"[1, 0, 0, 0, 0]",In dense and cold regions of the interstellar ...,"['dense', 'cold', 'region', 'interstellar', 'm..."
2,Gamma-ray lines in modern astrophysics,Gamma-ray astronomy provides a direct window i...,astrophysics,humano,"[1, 0, 0, 0, 0]",Gamma-ray astronomy provides a direct window i...,"['gamma-ray', 'astronomy', 'provides', 'direct..."
3,From Coulomb excitation cross sections to non-...,Coulomb and nuclear dissociation of $^{17}$Ne ...,astrophysics,humano,"[1, 0, 0, 0, 0]",Coulomb and nuclear dissociation of $^{17}$Ne ...,"['coulomb', 'nuclear', 'dissociation', 'ne', '..."
4,Astrophysical Black Holes: A Compact Pedagogic...,Black holes are among the most extreme objects...,astrophysics,humano,"[1, 0, 0, 0, 0]",Black holes are among the most extreme objects...,"['black', 'hole', 'among', 'extreme', 'object'..."
...,...,...,...,...,...,...,...
7745,Superconducting LaAlO3/SrTiO3 Nanowires,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe've achieved a groundbreaking breakthrou...,"[""'ve"", 'achieved', 'groundbreaking', 'breakth..."
7746,Coupling a single Nitrogen-Vacancy center to a...,Here's a rewritten version of the text in my o...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe propose a groundbreaking idea: connecti...,"['propose', 'groundbreaking', 'idea', 'connect..."
7747,Superconducting and Anti-Ferromagnetic Phases ...,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nThis study establishes a connection betwee...,"['study', 'establishes', 'connection', 'two', ..."
7748,New Superconductivity Dome in LaFeAsO$_{1-x}$F...,Here's the rewritten text:\n\nA fascinating ph...,superconductivity,llama3,"[0, 0, 0, 0, 1]",Here's the rewritten text:\n\nA fascinating ph...,"[""'s"", 'rewritten', 'text', 'fascinating', 'ph..."


In [20]:
import pandas as pd
from transformers import RobertaModel, RobertaTokenizer
import torch

# Cargar el tokenizador y el modelo pre-entrenado de RoBERTa-base
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
from tqdm import tqdm 
textos = datos['abstract_limpio'].tolist()
# Función para obtener los embeddings de las oraciones
def obtener_embeddings(oraciones):
    embeddings = []
    for oracion in tqdm(oraciones,desc="Procesando embeddings"):
        tokens = tokenizer(oracion, return_tensors="pt",truncation = True)
        with torch.no_grad():
            outputs = model(**tokens)
        embedding_oracion = torch.mean(outputs.last_hidden_state, dim=1)
        embeddings.append(embedding_oracion)
    return embeddings

# Obtener los embeddings de las oraciones
lista_embeddings = obtener_embeddings(textos)

Procesando embeddings: 100%|███████████████████████████████████████████████████████| 7750/7750 [23:06<00:00,  5.59it/s]


In [59]:
print(len(lista_embeddings))

7750


In [60]:
lista_embeddings[0]

tensor([[ 7.8024e-02,  1.9705e-01, -2.0739e-02,  9.4564e-02,  1.2824e-01,
          2.4060e-01,  6.1715e-02,  6.8355e-03, -3.3821e-02, -2.3071e-02,
         -1.4039e-01, -2.3197e-01,  1.2366e-01, -3.2405e-02,  1.0557e-01,
         -3.5048e-01,  6.0722e-03,  1.2401e-01, -4.0504e-02,  1.6840e-03,
         -4.1068e-02,  1.4175e-01, -6.7445e-02, -2.3354e-02, -5.9382e-02,
          6.5422e-02,  7.0820e-02, -3.9418e-02, -4.0954e-02, -5.0155e-02,
          1.2849e-02,  7.0372e-02,  1.9434e-01, -1.0157e-02, -1.5319e-01,
         -1.0247e-02, -1.3299e-01,  5.0092e-02,  1.1887e-01,  1.3433e-02,
         -2.3574e-03,  2.8852e-03,  3.4091e-02,  5.1075e-02, -4.0177e-02,
         -1.1713e-01, -1.2483e-01, -5.3945e-02,  9.5178e-02, -4.2143e-03,
          1.3563e-01, -7.1324e-02, -4.2744e-02, -2.2641e-02,  2.6882e-02,
          5.3104e-02,  9.4445e-02, -7.1072e-02,  2.6500e-02, -1.3424e-01,
         -8.7504e-02, -3.1570e-02,  6.5674e-03,  1.0987e-01,  5.9831e-02,
         -6.2705e-02,  2.3805e-02, -2.

In [69]:
lista_de_listas = [tensor.tolist() for tensor in lista_embeddings]
print(len(lista_de_listas))

lista_limpia = []
for i in range(len(lista_de_listas)):
    lista_limpia.append(lista_de_listas[i][0])

7750


In [72]:
df = pd.DataFrame(lista_limpia)
df
df.to_csv('roberta_embeddings.csv', index=False)  # Para CSV


In [78]:
df = pd.read_csv('roberta_embeddings.csv')
etiquetas = datos['cat_one_hot'].tolist()
df['cat_one_hot'] = etiquetas
df
df.to_csv('roberta_embeddings.csv', index=False)  # Para CSV
